- Add project names to SQL query
- Add user's first post date to SQL query
- Add "Product" to SQL query
- Change field names
    - Yellow: Add
    - Green: Change in code
    - Red: Leave alone
<!-- - Trim last three characters from adwire_opportunity_id -->
- Fix revenue_market_area values
- Remove pre-Feb 2nd (no rev info) rows)
- {}

In [1]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.DataFrame(pd.read_csv('March iHeart Everyone HR Pull.xlsx - Sheet1.csv'))
accounts = pd.DataFrame(pd.read_csv('fullmerge - account_names.csv'))
empl_roll = pd.DataFrame(pd.read_csv('CR Indexed Worker Roster - Sales 2021-01-04 19_16 CST.xlsx - Sheet1.csv'))
user_df = pd.DataFrame(pd.read_csv('ihm_users.csv'))
market_owners = pd.DataFrame(pd.read_csv('ihm_account_management.csv'))
workday = pd.DataFrame(pd.read_csv('usersChanges_2021_02_15_11_54.csv'))
revenue = pd.DataFrame(pd.read_csv('sf_duplicates.csv'))
projects = pd.DataFrame(pd.read_csv('iheart_projects.csv'))
newhr = pd.DataFrame(pd.read_csv('March iHeart Everyone HR Pull.xlsx - Sheet1.csv'))

In [3]:
projects.columns

Index(['project_id', 'create_ts', 'email', 'title', 'name', 'name.1', 'group',
       'submissions', 'opportynity_id', 'credits_used', 'industry', 'tone',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16'],
      dtype='object')

In [4]:
empl_roll.rename(columns = {'Worker': 'hr_worker', 'Job Title': 'hr_job_title', 'Business Title': 'hr_business_title',
                                       'Hire Date': 'hr_hire_date', 'Phone - Primary Work': 'hr_work_phone',
                                       'Mobile Phones': 'hr_mobile_phone', "Worker's Manager": 'hr_worker_manager',
                                        "Worker's Manager Business Title": "hr_worker_manager_business_title",
                                       "Worker's Manager's Manager": "hr_worker_manager_manager",
                                       "Worker's Manager's Manager Business Title": "hr_worker_manager_manager_business_title",
                                       'Market': 'hr_market', 'Market President': 'hr_market_president', 'Region/Area': 'hr_region_area',
                                       'Area/Region/Community President': 'hr_area_region_community_president', 'Division': 'hr_division',
                                       'Division President': 'hr_division_president'}, inplace=True)

user_df.rename(columns = {'client': 'adwire_client', 'group_x': 'adwire_group', 'name_x': 'adwire_name_x',
                                       'email': 'adwire_email', 'projects': 'adwire_projects', 'spend': 'adwire_spend',
                                       'project_ts': 'adwire_first_posting', 'group_cap': 'adwire_cap', 'owner': 'account_owner',
                                        'group_upper': 'adwire_group_upper'}, inplace=True)

workday.rename(columns = {'first name_x': 'workday_first_name', 
                                       'action': 'workday_action', 'client code': 'workday_client_code', 'first name_y': 'workday_first_name2',
                                       'last name': 'workday_last_name', 'group_y': 'workday_group_y'}, inplace=True)

revenue.rename(columns = {'Opportunity: Opportunity ID': 'revenue_opportunity_id', 'stage': 'revenue_stage',
                                        'market_division': 'revenue_market_division', 'market_area': 'revenue_market_area', 'market': 'revenue_market',
                                       'ae_name': 'revenue_ae_name', 'new_business': 'revenue_new_business', 'account': 'revenue_account',
                                       'opportunity': 'revenue_opportunity', 'project_id': 'project_id', 'form_date': 'revenue_form_date',
                                       'opportunity_created_date': 'revenue_opportunity_created_date', 'opportunity_closed_date': 'revenue_opportunity_closed_date',
                                       'gross_expected_revenue': 'revenue_gross_expected_revenue', 'gross_amount': 'revenue_gross_amount',
                                        'form_created_on_closed_opp': 'revenue_form_created_on_closed_opp',
                                       'market_type': 'revenue_market_type'}, inplace=True)

projects.rename(columns = {'create_ts': 'adwire_post_date', 'name_y': 'adwire_name_y', 'group': 'adwire_group2',
                                        'submissions': 'adwire_submissions', 'opportynity_id': 'adwire_opportunity_id',
                                       'credits_used': 'adwire_credits_used', 'industry': 'adwire_industry', 'tone': 'adwire_tone'}, inplace=True)

In [5]:
newhr['User - Division'] = ['iHeart Media - ' + x.replace(' - Market', '') for x in newhr['User - Division']]

empl_roll.hr_market = ['iHeart Media - ' + x.replace(' - Market', '') for x in empl_roll.hr_market]

user_df.group = ['iHeart Media - ' + x.replace(' - Market', '') for x in user_df.group]

workday.group = ['iHeart Media - ' + str(x) for x in workday.group]

projects.adwire_group2 = ['iHeart Media - ' + str(x) for x in projects.adwire_group2]

projects.adwire_group2 = projects.adwire_group2.str.strip()

In [6]:
projects.loc[(projects.adwire_group2 == 'iHeart Media - St. Petersburg, FL | Inside Sales'), 'adwire_group2'] = 'iHeart Media - Inside Sales'
projects.loc[(projects.adwire_group2 == 'iHeart Media - Modesto / Stockton, CA'), 'adwire_group2'] = 'iHeart Media - Modesto/Stockton, CA'
projects.loc[(projects.adwire_group2 == 'iHeart Media - Roanoke / Lynchburg, VA'), 'adwire_group2'] = 'iHeart Media - Roanoke/Lynchburg, VA'
projects.loc[(projects.adwire_group2 == 'iHeart Media - Lexington / Somerset, KY'), 'adwire_group2'] = 'iHeart Media - Lexington/Somerset, KY'

In [7]:
revenue.revenue_market = revenue.revenue_market.astype(str)

c = [x[:-2] for x in revenue.revenue_market]
d = [x[-2:] for x in revenue.revenue_market]

f = ['iHeart Media - '+ i + ', ' + j for i, j in zip(c, d)]

df5 = revenue

df5['revenue_account'] = pd.Series(f)

df5.loc[(df5['revenue_account'] == 'iHeart Media - Inside Sal, es'),'revenue_account'] = 'iHeart Media - Inside Sales'
df5.loc[(df5['revenue_account'] == 'iHeart Media - WestPalmBeach, FL'), 'revenue_account'] = 'iHeart Media - West Palm Beach, FL'
df5.loc[(df5['revenue_account'] == 'iHeart Media - CorpusChristi, TX'), 'revenue_account'] = 'iHeart Media - Corpus Christi, TX'
df5.loc[(df5['revenue_account'] == 'iHeart Media - SanAntonio, TX'), 'revenue_account'] = 'iHeart Media - San Antonio, TX'
df5.loc[(df5['revenue_account'] == 'iHeart Media - ElPaso, TX'), 'revenue_account'] = 'iHeart Media - El Paso, TX'
df5.loc[(df5['revenue_account'] == 'iHeart Media - NewHaven, CT'), 'revenue_account'] = 'iHeart Media - New Haven, CT'
df5.loc[(df5['revenue_account'] == 'iHeart Media - FortMyers, FL'), 'revenue_account'] = 'iHeart Media - Fort Myers/Naples, FL'
df5.loc[(df5['revenue_account'] == 'iHeart Media - SiouxCity, IA'), 'revenue_account'] = 'iHeart Media - Sioux City, IA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - SaltLakeCity, UT'), 'revenue_account'] = 'iHeart Media - Salt Lake City, UT'
df5.loc[(df5['revenue_account'] == 'iHeart Media - MyrtleBeach, SC'), 'revenue_account'] = 'iHeart Media - Myrtle Beach, SC'
df5.loc[(df5['revenue_account'] == 'iHeart Media - FortCollins, CO'), 'revenue_account'] = 'iHeart Media - Fort Collins, CO'
df5.loc[(df5['revenue_account'] == 'iHeart Media - LosAngeles, CA'), 'revenue_account'] = 'iHeart Media - Los Angeles, CA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - ColoradoSprings, CO'), 'revenue_account'] = 'iHeart Media - Colorado Springs, CO'
df5.loc[(df5['revenue_account'] == 'iHeart Media - NewOrleans, LA'), 'revenue_account'] = 'iHeart Media - New Orleans, LA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - BatonRouge, LA'), 'revenue_account'] = 'iHeart Media - Baton Rouge, LA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - GrandForks, ND'), 'revenue_account'] = 'iHeart Media - El Paso, TX'
df5.loc[(df5['revenue_account'] == 'iHeart Media - LasVegas, NV'), 'revenue_account'] = 'iHeart Media - Las Vegas, NV'
df5.loc[(df5['revenue_account'] == 'iHeart Media - CedarRapids, IA'), 'revenue_account'] = 'iHeart Media - Cedar Rapids/Iowa City, IA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - CapeCod, MA'), 'revenue_account'] = 'iHeart Media - Cape Cod, MA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - FortSmith, AR'), 'revenue_account'] = 'iHeart Media - Fort Smith, AR'
df5.loc[(df5['revenue_account'] == 'iHeart Media - GrandRapids, MI'), 'revenue_account'] = 'iHeart Media - Grand Rapids, MI'
df5.loc[(df5['revenue_account'] == 'iHeart Media - DesMoines, IA'), 'revenue_account'] = 'iHeart Media - Des Moines, IA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - SaintLouis, MO'), 'revenue_account'] = 'iHeart Media - Saint Louis, MO'
df5.loc[(df5['revenue_account'] == 'iHeart Media - BayArea, CA'), 'revenue_account'] = 'iHeart Media - San Francisco, CA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - PanamaCity, FL'), 'revenue_account'] = 'iHeart Media - Panama City/Tallahassee, FL'
df5.loc[(df5['revenue_account'] == 'iHeart Media - OklahomaCity, OK'), 'revenue_account'] = 'iHeart Media - Oklahoma City, OK'
df5.loc[(df5['revenue_account'] == 'iHeart Media - n, an'), 'revenue_account'] = 'iHeart Media - No Market'
df5.loc[(df5['revenue_account'] == 'iHeart Media - EauClaire, WI'), 'revenue_account'] = 'iHeart Media - Eau Claire, WI'
df5.loc[(df5['revenue_account'] == 'iHeart Media - SanDiego, CA'), 'revenue_account'] = 'iHeart Media - San Diego, CA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - LittleRock, AR'), 'revenue_account'] = 'iHeart Media - Little Rock, AR'
df5.loc[(df5['revenue_account'] == 'iHeart Media - El Paso, TX'), 'revenue_account'] = 'iHeart Media - El Paso, TX'
df5.loc[(df5['revenue_account'] == 'iHeart Media - Roanoke, VA'), 'revenue_account'] = 'iHeart Media - Roanoke/Lynchburg, VA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - Sarasota, FL'), 'revenue_account'] = 'iHeart Media - Sarasota/Bradenton, FL'
df5.loc[(df5['revenue_account'] == 'iHeart Media - Ashland, OH'),'revenue_account'] = 'iHeart Media - Ashland/Mansfield, OH'
df5.loc[(df5['revenue_account'] == 'iHeart Media - NewYork, NY'),'revenue_account'] = 'iHeart Media - New York City, NY'
df5.loc[(df5['revenue_account'] == 'iHeart Media - Modesto, CA'),'revenue_account'] = 'iHeart Media - Modesto/Stockton, CA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - Hattiesburg, MS'),'revenue_account'] = 'iHeart Media - Laurel, MS'
df5.loc[(df5['revenue_account'] == 'iHeart Media - Albany, GA'),'revenue_account'] = 'iHeart Media - Columbus, GA'
df5.loc[(df5['revenue_account'] == 'iHeart Media - St Petersburg , FL'),'revenue_account'] = 'iHeart Media - Inside Sales'

# df6.fillna('No Data', inplace = True)

In [ ]:
proj

In [8]:
accountnames = accounts.Accounts.unique().tolist()

hrnames = newhr['User - Division'].unique().tolist()

empl_markets = empl_roll.hr_market.unique().tolist()

user_df_groups = user_df.group.unique().tolist()

workday_groups = workday.group.unique().tolist()

revenue_markets = revenue.revenue_account.unique().tolist()

projects_groups = projects.adwire_group2.unique().tolist()

In [9]:
def Differences(hrnames, accountnames):
    li_diff = [i for i in hrnames + accountnames if i not in hrnames or i not in accountnames]
    missingfromhr = [i for i in hrnames + accountnames if i not in hrnames]
    missingfromaccountnames = [i for i in hrnames + accountnames if i not in accountnames]
    return 'combined differences:', li_diff, 'missing from table:', missingfromhr, 'missing from accountnames:', missingfromaccountnames

# Differences(hrnames, accountnames)

In [10]:
merge = pd.merge(projects, revenue, on='project_id', how='left')

In [11]:
revenue.to_csv('sf_duplicates_v2.csv')

projects.to_csv('projects_v2.csv')

In [12]:
merge.revenue_account = merge.revenue_account.astype(str)
merge.revenue_account = np.where(merge.revenue_account == 'nan', merge.adwire_group2, merge.revenue_account)
# Possible flag: adwire_group2 may be missattribution; creator's market is not necessarily opp owner's market
merge.revenue_account = np.where(merge.revenue_account == 'iHeart Media - No Market', merge.adwire_group2, merge.revenue_account)

In [35]:
Differences(merge.adwire_group2.unique().tolist(), accountnames)

('combined differences:',
 ['iHeart Media - Sam Hensley',
  'iHeart Media - McAllen, TX',
  'iHeart Media - bulk custom food orders',
  'iHeart Media - IHM Innovation Lab',
  'iHeart Media - 9',
  'iHeart Media - Corporate Group',
  'iHeart Media - 6',
  'iHeart Media - AdWire Draft Scripts | 15 AND 30 Seconds',
  'iHeart Media - Helen Lazaro',
  'iHeart Media - Andrew Gonzalez',
  'iHeart Media - AdWire Final Scripts | 60 Seconds',
  'iHeart Media - Dave Sonefeld',
  'iHeart Media - Libby Hogan',
  'iHeart Media - Auto Team',
  'iHeart Media - Centralized Digital',
  'iHeart Media - Design Hub',
  'iHeart Media - Govt Initiatives',
  'iHeart Media - Integrated Revenue Strategies',
  'iHeart Media - Lexington, KY',
  'iHeart Media - Markets Events Team',
  'iHeart Media - Mcallen, TX',
  'iHeart Media - Revenue Management',
  'iHeart Media - Sales Operation',
  'iHeart Media - Somerset, KY',
  'iHeart Media - Tallahassee, FL'],
 'missing from table:',
 ['iHeart Media - Auto Team',
  'i

In [14]:
merge.to_csv('revenue_projects_merge.csv')
projects.to_csv('projects.csv')
revenue.to_csv('revenue.csv')

In [ ]:
merge['Opportunity Record Type'] == '0125G000000uFriQAE'

In [27]:
merge.rename(columns = {'opportunity_id': 'iHeart Opportunity ID', 'revenue_stage': 'Stage',
                       'revenue_market_division': 'Revenue Market Division', 'revenue_market_area': 'Revenue Market Area',
                       'revenue_opportunity': 'Opportunity Name', 'revenue_opportunity_created_date': 'Created Date',
                       'revenue_opportunity_close_date': 'Close Date', 'revenue_gross_amount': 'Amount',
                       'revenue_form_created_on_closed_opp': 'Created on Closed Opp', 'revenue_market_type': 'Revenue Market Type',
                        'project_id': 'AdWire Project ID', 'email': ''
                       }, inplace=True)

In [25]:
merge.columns

Index(['project_id', 'adwire_post_date', 'email', 'title', 'name', 'name.1',
       'adwire_group2', 'adwire_submissions', 'adwire_opportunity_id',
       'adwire_credits_used', 'adwire_industry', 'adwire_tone', 'Unnamed: 12',
       'Unnamed: 16', 'iHeart Opportunity ID', 'Stage',
       'Revenue Market Division', 'Revenue Market Area', 'revenue_market',
       'revenue_ae_name', 'revenue_new_business', 'revenue_account',
       'Opportunity Name', 'revenue_form_date', 'Created Date',
       'revenue_opportunity_closed_date', 'revenue_gross_expected_revenue',
       'iHeart Gross Revenue', 'Created on Closed Opp', 'Type'],
      dtype='object')

In [29]:
merge[5000:5001]

,project_id,adwire_post_date,email,title,name,name.1,adwire_group2,adwire_submissions,adwire_opportunity_id,adwire_credits_used,adwire_industry,adwire_tone,Unnamed: 12,Unnamed: 16,iHeart Opportunity ID,Stage,Revenue Market Division,Revenue Market Area,revenue_market,revenue_ae_name,revenue_new_business,revenue_account,Opportunity Name,revenue_form_date,Created Date,revenue_opportunity_closed_date,revenue_gross_expected_revenue,iHeart Gross Revenue,Created on Closed Opp,Type
5000,Akys4C9AAAA,2020-04-21 11:07:36,SarahSteckler@iheartmedia.com,Christian Chrysler Dodge Jeep Ram | Christian ...,AdWire Draft Scripts | 30 Seconds,Sarah Steckler,"iHeart Media - Grand Forks, ND",6,0060y00001DdWMPAA3,1,Auto/Recreational Vehicles & Service,I don't care as long as it's good,NaN,NaN,0060y00001DdWMP,Closed - Won,Lankford,MidNorth,GrandForksND,Sarah Steckler,Existing Business,"iHeart Media - El Paso, TX","Christian-RAM April 2020 Incentive-Steckler, S...",20200421.0,4/21/2020,4/22/2020,555.0,555.0,0.0,Community


In [ ]:
merge.drop(['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'adwire_post_date',], axis=1, inplace=True)

In [33]:
merge.columns

Index(['project_id', 'adwire_post_date', 'email', 'title', 'name', 'name.1',
       'adwire_group2', 'adwire_submissions', 'adwire_opportunity_id',
       'adwire_credits_used', 'adwire_industry', 'adwire_tone', 'Unnamed: 12',
       'Unnamed: 16', 'iHeart Opportunity ID', 'Stage',
       'Revenue Market Division', 'Revenue Market Area', 'revenue_market',
       'revenue_ae_name', 'revenue_new_business', 'revenue_account',
       'Opportunity Name', 'revenue_form_date', 'Created Date',
       'revenue_opportunity_closed_date', 'revenue_gross_expected_revenue',
       'iHeart Gross Revenue', 'Created on Closed Opp', 'Type'],
      dtype='object')